<a href="https://colab.research.google.com/github/invi-bhagyesh/study-GPT/blob/main/training_tess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/invi-bhagyesh/study-GPT.git

Cloning into 'study-GPT'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 109 (delta 28), reused 98 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 4.40 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd drive/MyDrive/LLM/study-GPT

/content/drive/MyDrive/LLM/study-GPT


In [4]:
%ls

assets/  LICENSE  model/     requirements.txt  train.py
data/    main.py  README.md  training.ipynb    wandb/


In [5]:
# Optional: install wandb if you want
# !pip install wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import math
from tqdm import tqdm

# Optional
# import wandb

In [6]:
from model.gpt2 import GPT2


In [10]:
# For GPT-2
from transformers import GPT2Config
from model.gpt2 import GPT2  # assuming you have your custom GPT2 implementation

# Create config
config = GPT2Config(
    n_layer=12,    # number of transformer layers
    n_head=12,     # number of attention heads
    n_embd=768    # embedding dimension
)

# Initialize model
model = GPT2(config)

# 1. Basic parameter count
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters
total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

Total trainable parameters: 124,439,808


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [8]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

tokenized_dataset.save_to_disk("data/tokenized_wikitext2")

train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]



Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36718 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})

In [15]:
total_tokens = sum([len(tokenizer(x['text'])['input_ids']) for x in dataset['train']])
print(total_tokens)

2391884


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model.config import GPT2Config
from model.gpt2 import GPT2
from transformers import GPT2Tokenizer
from tqdm.notebook import tqdm   # NOTE: notebook-friendly tqdm
import wandb

In [13]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: invi-bhagyesh (invi-bhagyesh-manipal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
from train import train_gpt2

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_loader = DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator)


ModuleNotFoundError: No module named 'datasets'

In [ ]:
"""train_gpt2(train_loader, val_loader, n_epochs=3)
# Save the model after training
torch.save(model.state_dict(), "data/gpt2_pretrained_invi.pth")
print("Model saved to data/gpt2_pretrained_invi.pth")"""

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: invi-bhagyesh (invi-bhagyesh-manipal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/3:   0%|          | 1/4590 [01:30<115:22:02, 90.50s/it]

In [2]:
train_gpt2(
    train_loader,
    val_loader,
    n_epochs=3,
    checkpoint_path="model_weight_1/gpt2_epoch_epoch2.pt"
)

NameError: name 'train_gpt2' is not defined

In [11]:
import torch
from model.config import GPT2Config
from model.gpt2 import GPT2

# Load the model config
config = GPT2Config()
model = GPT2(config)

# Load checkpoint
checkpoint_path = "model_weight_1/gpt2_epoch_epoch2.pt"
checkpoint = torch.load(checkpoint_path, map_location="cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(checkpoint)

print("✅ Model checkpoint loaded successfully!")

KeyboardInterrupt: 

In [23]:
model.to("cpu")

GPT2(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (c_attn): Linear(in_features=768, out_features=2304, bias=True)
        (c_proj): Linear(in_features=768, out_features=768, bias=True)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=3072, out_features=768, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [14]:
from model.config import GPT2Config

In [24]:
import torch
from transformers import GPT2Tokenizer

# Load tokenizer (use same tokenizer used for training)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def generate_text(model, input_text, max_length=50, device="cuda"):
    model.eval()  # Set to evaluation mode
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Greedy decoding
    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids)  # Forward pass
            next_token = torch.argmax(logits[:, -1, :], dim=-1)  # Get most likely next token
            input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=-1)  # Append to input

            if next_token.item() == tokenizer.eos_token_id:  # Stop if EOS token is generated
                break

    return tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)

# Load your model
from model.gpt2 import GPT2  # Ensure correct import
config = GPT2Config()  # Load same config used during training
model = GPT2(config)
model.load_state_dict(torch.load("model_weight_1/gpt2_epoch_epoch2.pt", map_location=torch.device('cpu')))
model.to("cpu")

# **Run Inference**
input_text = "Once upon a time"
output_text = generate_text(model, input_text)
print(output_text)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [28]:
from model.gpt2 import GPT2

In [29]:
# Ensure model is on CPU
model.to("cpu")

# Function to generate text
def generate_text(model, input_text):
    model.eval()  # Set model to evaluation mode
    input_ids = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0).to("cpu")  # Ensure input is on CPU
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=50)  # Adjust length as needed
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Run Inference
input_text = "Once upon a time"
output_text = generate_text(model, input_text)
print(output_text)

AttributeError: 'GPT2' object has no attribute 'generate'

In [30]:
print(hasattr(model, "generate"))  # Should print True

False


In [31]:
model = GPT2(config)  # Ensure you're using the correct GPT2 class